<b>

 <h1><center>Homework 3 - Building NLP Pipeline</center></h1>
    <center><b>Problem</b> : Quora Insincere Questions Classification -
Detect toxic content to improve online conversations.</center>
<center><b>Details</b> : We are given a dataset of word sequences with target labels 1(insincere) and 0(sincere). Our goal is to train nlp model with neural networks which will have decent accuracy. Word embeddings are also provided. </center>

<h3>Imports<h3>

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import re
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly import subplots
import plotly.offline as py
from wordcloud import WordCloud, STOPWORDS
from collections import defaultdict
import operator 
import os
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer
import torch
import torch.nn as nn
from torchtext.legacy import data
import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import torch.nn.functional as F
import zipfile

<h2>Part 1: Read and Analyse Data</h2>

In [ ]:
train_df = pd.read_csv("../input/quora-insincere-questions-classification/train.csv")
test_df = pd.read_csv("../input/quora-insincere-questions-classification/test.csv")

In [ ]:
print('Train data dimension: ', train_df.shape)
train_df.head()

As we can see we are dealing with sequential input which has one label output, therefore we are going to use rnn sentiment analys technique which fits best to this problem(According to my current knowledge of NLP :D).

In [ ]:
train_df.isna().sum()

As we know from the problem description, our sequential inputs are questions, so let's check how many of these inputs has no question symbol in it.

In [ ]:
not_contains_question_symbols = train_df[~train_df['question_text'].str.contains(pat = "\?")]
print("Count - ",not_contains_question_symbols["question_text"].count())
print(not_contains_question_symbols["question_text"][:40].to_string())

del not_contains_question_symbols

We got only 300 records here and as it's shown above, most of these records have question format, so we will leave them untouched.

Let's also check the distribution of sentence lengths to determine the mean one.

In [ ]:
train_df['word_count'] = train_df['question_text'].apply(lambda x: len(x.split()))
train_df.pivot(columns='target', values='word_count').plot.hist(bins=range(0,60))

In [ ]:
cnt_srs = train_df['target'].value_counts()
labels = (np.array(cnt_srs.index))
sizes = (np.array((cnt_srs / cnt_srs.sum())*100))

trace = go.Pie(labels=labels, values=sizes)
layout = go.Layout(
    title='Target distribution',
    font=dict(size=18),
    width=600,
    height=600,
)
data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="usertype")

print("Label 1 count: ", train_df[train_df['target'] == 1].count()[0])
print("Label 0 count: ", train_df[train_df['target'] == 0].count()[0])

del cnt_srs, labels, sizes, trace, layout, data, fig

Unfortunately given data has very unbalanced distribution of targets, target 0 has way more observed data...

In [ ]:
max_seq_len = round(sum(train_df['word_count'])/len(train_df))
print('Average word count in questions is ', max_seq_len)

We will pad or trim all sequences to **max_seq_len** length later in data processing stage.

In [ ]:
print('Max word length of questions in train is {0:.0f}.'.format(np.max(train_df['question_text'].apply(lambda x: len(x.split())))))

<h2><center>Word Cloud of Question Words</center></h2>

In [ ]:
def plot_wordcloud(text, mask=None, max_words=200, max_font_size=100, figure_size=(24.0,16.0), 
                   title = None, title_size=40, image_color=False):
    stopwords = set(STOPWORDS)
    more_stopwords = {'one', 'br', 'Po', 'th', 'sayi', 'fo', 'Unknown'}
    stopwords = stopwords.union(more_stopwords)

    wordcloud = WordCloud(background_color='black',
                    stopwords = stopwords,
                    max_words = max_words,
                    max_font_size = max_font_size, 
                    random_state = 42,
                    width=800, 
                    height=400,
                    mask = mask)
    wordcloud.generate(str(text))
    
    plt.figure(figsize=figure_size)
    if image_color:
        image_colors = ImageColorGenerator(mask);
        plt.imshow(wordcloud.recolor(color_func=image_colors), interpolation="bilinear");
        plt.title(title, fontdict={'size': title_size,  
                                  'verticalalignment': 'bottom'})
    else:
        plt.imshow(wordcloud);
        plt.title(title, fontdict={'size': title_size, 'color': 'black', 
                                  'verticalalignment': 'bottom'})
    plt.axis('off');
    plt.tight_layout()  
    
    del stopwords, more_stopwords, wordcloud
    
plot_wordcloud(train_df["question_text"], title="Word Cloud of Questions")

<center><h2>Word Frequency plot of sincere and insincere questions</h2></center>

In [ ]:
train1_df = train_df[train_df["target"]==1]
train0_df = train_df[train_df["target"]==0]

## custom function for ngram generation ##
def generate_ngrams(text, n_gram=1):
    token = [token for token in text.lower().split(" ") if token != "" if token not in STOPWORDS]
    ngrams = zip(*[token[i:] for i in range(n_gram)])
    return [" ".join(ngram) for ngram in ngrams]

## custom function for horizontal bar chart ##
def horizontal_bar_chart(df, color):
    trace = go.Bar(
        y=df["word"].values[::-1],
        x=df["wordcount"].values[::-1],
        showlegend=False,
        orientation = 'h',
        marker=dict(
            color=color,
        ),
    )
    return trace

## Get the bar chart from sincere questions ##
freq_dict = defaultdict(int)
for sent in train0_df["question_text"]:
    for word in generate_ngrams(sent):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace0 = horizontal_bar_chart(fd_sorted.head(50), 'blue')

## Get the bar chart from insincere questions ##
freq_dict = defaultdict(int)
for sent in train1_df["question_text"]:
    for word in generate_ngrams(sent):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace1 = horizontal_bar_chart(fd_sorted.head(50), 'blue')

# Creating two subplots
fig = subplots.make_subplots(rows=1, cols=2, vertical_spacing=0.04,
                          subplot_titles=["Frequent words of sincere questions", 
                                          "Frequent words of insincere questions"])
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig['layout'].update(height=1200, width=900, paper_bgcolor='rgb(233,233,233)', title="Word Count Plots")
py.iplot(fig, filename='word-plots')

del train1_df, train0_df, freq_dict, fd_sorted, trace0, trace1

**Observations**:

1. Both sides share some top words for example 'will', 'people', 'think'
2. 'trump' is on the second place of insincere words...


<h2>Part 2: Data Preprocessig</h2>
<p>We need to process data to match embeddings dictionary words with high accuracy.</p>
<h3>Let's define some helper functions:</h3>

In [ ]:
def build_vocab(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = {}
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

def check_coverage(vocab,embeddings_index):
    a = {}
    oov = {}
    k = 0
    i = 0
    for word in tqdm(vocab):
        try:
            a[word] = embeddings_index[word]
            k += vocab[word]
        except:

            oov[word] = vocab[word]
            i += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(a) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(k / (k + i)))
    sorted_x = sorted(oov.items(), key=operator.itemgetter(1))[::-1]

    return sorted_x

Let's start with fixing common misspellings and making words lowercase.

In [ ]:
mispell_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'}

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispellings, mispellings_re = _get_mispell(mispell_dict)
def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace, text)

train_df["question_text"] = train_df["question_text"].str.lower()
test_df["question_text"] = test_df["question_text"].str.lower()

train_df["question_text"] = train_df["question_text"].apply(lambda x: replace_typical_misspell(x))
test_df["question_text"] = test_df["question_text"].apply(lambda x: replace_typical_misspell(x))


Now, let's handle punctuations.

In [ ]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]
def clean_text(x):
    x = str(x)
    for punct in "/-'":
        x = x.replace(punct, ' ')
    for punct in '&':
        x = x.replace(punct, f' {punct} ')
    for punct in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~' + '“”’':
        x = x.replace(punct, '')
    return x

train_df["question_text"] = train_df["question_text"].apply(clean_text)
test_df["question_text"] = test_df["question_text"].apply(clean_text)


And we need to deal with numbers also.

In [ ]:
def clean_numbers(x):

    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

train_df["question_text"] = train_df["question_text"].apply(clean_numbers)
test_df["question_text"] = test_df["question_text"].apply(clean_numbers)

<h3>Building vocabulary</h3>


In [ ]:
def load_embed(file):
    def get_coefs(word,*arr): 
        return word, np.asarray(arr, dtype='float32')
    
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file, encoding='latin'))
        
    return embeddings_index

In [ ]:
vocab = build_vocab(train_df["question_text"].progress_apply(lambda x: x.split()))
print("We have", len(vocab), "unique words!")

Extracting embeddings.zip and loading glove embeddings.

In [ ]:
# os.makedirs('./embeddings')
with zipfile.ZipFile('../input/quora-insincere-questions-classification/embeddings.zip', 'r') as zip_ref:
    zip_ref.extract('glove.840B.300d/glove.840B.300d.txt', './embeddings')
glove = './embeddings/glove.840B.300d/glove.840B.300d.txt'
word_embeddings = load_embed(glove)

In [ ]:
oov = check_coverage(vocab,word_embeddings)

In [ ]:
print(len(oov))

From the results we can see that 99% of words in our data has it's embedding and that's a great result. We may assume that words in oov are not very common and can't be handled generally.

In [ ]:
print(oov[:30])
del oov

In [ ]:
unknown_embedding = torch.zeros_like(torch.empty(300))
vocab_embeddings = {'<pad>': unknown_embedding}
for word in vocab:
    try:
        vocab_embeddings[word] = torch.tensor(word_embeddings[word])
    except :
        vocab_embeddings[word] = unknown_embedding
del word_embeddings, vocab

#  Split data to valid and train, pad/trim to maxlen and then batchify it

In [ ]:
train_df = train_df[['question_text', 'target']]
o_class = train_df.loc[train_df.target == 0, :]
l_class = train_df.loc[train_df.target == 1, :]

valid_o = o_class.iloc[:10000, :]
valid_l = l_class.iloc[:10000, :]

train_o = o_class.iloc[10000:, :]
train_l = l_class.iloc[10000:, :]

train = pd.concat([train_o, train_l], axis=0)
valid = pd.concat([valid_o, valid_l], axis=0)

del train_df, o_class, l_class

In [ ]:
train = list(train.to_records(index=False))
valid = list(valid.to_records(index=False))

In [ ]:
def collate_batch(batch):
    label_list, text_list = [], []
    for text, label in batch:
        label_list.append(label)
        splitted = text.split()
        if max_seq_len < len(splitted):
            splitted = splitted[:max_seq_len]
        elif len(splitted) < max_seq_len:
            splitted += ['<pad>'] * (max_seq_len - len(splitted))
        text_list.append(torch.stack(list(map(lambda x : vocab_embeddings[x], splitted))))
    label_list = torch.tensor(label_list, dtype=torch.int32)
    text_list = torch.stack(text_list, dim=0)
    return text_list, label_list

from torch.utils.data import DataLoader

batch_size=100

train_dl = DataLoader(train, 
                      batch_size=batch_size, 
                      shuffle=True, 
                      collate_fn=collate_batch)

valid_dl = DataLoader(valid, 
                      batch_size=batch_size*2, 
                      shuffle=False, 
                      collate_fn=collate_batch)

<h2>Part 3: Model Training</h2>
We will define our Lstm Rnn Sentiment Analysis Model and train it on preprocessed data.

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.05):
        super(LSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        # We use LSTM becaues it solves vanishing gradient problem which occurs with simple RNN network
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout) # Data is very imbalanced, therefore to avoid overfitting we use dropout technique
        # We need one additional layer for final output
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        _, (hn, cn) = self.lstm(x, (h0,c0))
        
        out = self.relu(hn[1]) # Apply activation function 
        out = self.dropout(out) # Dropout Layer
        
        return self.fc(out)


**Confusion Matrix Plot Method**

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

**Hyper Parameters:**

In [ ]:
EMBEDDING_DIM = 300
HIDDEN_DIM = 128
SEQUENCE_LEN = max_seq_len
HIDDEN_LAYERS = 2

NUM_EPOCHS = 3
LEARNING_RATE = 0.001
NUM_CLASSES = 2
BATCH_SIZE = batch_size

**Model, LossFunction, Optimizer:**

In [ ]:
model = LSTM(EMBEDDING_DIM, HIDDEN_DIM, HIDDEN_LAYERS, NUM_CLASSES)
loss_fn = nn.CrossEntropyLoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)  

<h3>Train Loop</h3>

In [ ]:
def calc_accuracy(model: nn.Module):
    model.eval()

    Y, Y_PRED, Y_VAL = [], [], []
    with torch.no_grad():
        for i, (x, y) in enumerate(valid_dl):
            y_pred = model(x)

            Y.extend(y.tolist())
            Y_PRED.extend(y_pred.argmax(dim=1).tolist())
            Y_VAL.append(y_pred.cpu())

    cnf_matrix = confusion_matrix(Y, Y_PRED)

    print('Epoch Valid Loss: ', torch.nn.functional.cross_entropy(torch.cat(Y_VAL, dim=0), torch.LongTensor(Y)).item())

    plt.figure(figsize=(5, 5))
    plot_confusion_matrix(cnf_matrix, 
                        classes=['Neg', 'Pos'],
                        title='Confusion Matrix')

    plt.show()

def train():
    for epoch in range(NUM_EPOCHS):
        for i, (x, y) in enumerate(train_dl):
            model.train()
            optimizer.zero_grad()
            y_pred = model(x)
            loss = loss_fn(y_pred, y.long())
            loss.backward()
            optimizer.step()

            if (i + 1) % 1000 == 0:
                print(f'Epoch - {epoch + 1}, Iteration - {i + 1}, Train Loss {loss.item()}')
        calc_accuracy(model)

In [ ]:
train()

**Sample Predictions**


In [ ]:
def clean(sentence):
    sentence = sentence.lower()
    sentence = replace_typical_misspell(sentence)
    sentence = clean_text(sentence)
    sentence = clean_numbers(sentence)
    return sentence

def sentenceToEmbeddings(sentence):
    tokens = []
    for token in clean(sentence).split():
        try:
            tokens.append(vocab_embeddings[token])
        except:
            tokens.append(unknown_embedding)
    return torch.stack(tokens).unsqueeze(0)

def predict_sentiment(sentence):
    try:
        tensor = sentenceToEmbeddings(sentence)
    except:
        return 1 # if sentence is badly formatted or blank, it surely is insincere 
    predicted = model(tensor)
    return predicted.argmax(dim=1).item()

**Sincere Prediction**

In [ ]:
sincere_sentence = 'hello, i am fat. How can i lose some weight?'
print(predict_sentiment(sincere_sentence))

**Insincere Prediction**

In [ ]:
sincere_sentence = 'Fuck you mate!'
print(predict_sentiment(sincere_sentence))

<h2>Part 4: Save Submission.csv</h2>

In [ ]:
test_df.head()

In [ ]:
test_df.count()

In [ ]:
test_df['prediction'] = test_df['question_text'].apply(predict_sentiment) 

In [ ]:
test_df = test_df[['qid', 'prediction']]
test_df.groupby('prediction').count()

In [ ]:
test_df.to_csv('submission.csv',index=False)